# AI Comic Book Generator - Colab Launcher

This notebook runs the AI Comic Generator **completely FREE** using local Ollama models.
**No API keys needed!** Everything runs on Colab's free GPU.

In [ ]:
# @title 1. Setup Environment & Install Ollama
# @markdown Enter your GitHub repo URL and branch.
REPO_URL = "https://github.com/yogesh-dixit-dev/comic-generator.git" # @param {type:"string"}
BRANCH = "master" # @param {type:"string"}

import os
import sys

# Clone or update repo
if not os.path.exists("comic-gen"):
    !git clone -b $BRANCH $REPO_URL comic-gen
    %cd comic-gen
else:
    %cd /content/comic-gen
    !git pull

# Ensure requirements are installed
!pip install -q -r requirements.txt

# Install zstd (required by Ollama install script)
print("📦 Installing dependencies (zstd)...")
!apt-get update -qq && apt-get install -y -qq zstd

# Install Ollama
print("📦 Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server in background
print("🚀 Starting Ollama server...")
!nohup ollama serve > ollama.log 2>&1 &

# Pull the model
import time
time.sleep(5) # Wait for server
print("⬇️ Downloading Llama 3.2 model (this may take a few minutes)...")
!ollama pull llama3.2

print("\n✅ Environment Set Up! No API keys needed.")

In [ ]:
# @title 2. Configuration (Optional - Hugging Face Storage)
# @markdown Only needed if you want to save outputs to Hugging Face
import os
from google.colab import userdata

# Try getting HF token from Colab Secrets
try:
    os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')
    print("✅ Loaded HF token from Colab Secrets")
except:
    print("⚠️ No HF_TOKEN in Colab Secrets. Output will stay local.")
    pass

In [ ]:
# @title 3. Run Pipeline (100% Local - No API Calls!)
# @markdown Upload your story file to the 'comic-gen' folder using the file browser on the left.
INPUT_FILE = "story.txt" # @param {type:"string"}
OUTPUT_DIR = "output" # @param {type:"string"}
USE_HF_STORAGE = False # @param {type:"boolean"}
HF_REPO_ID = "your-username/comic-dataset" # @param {type:"string"}

# Make sure we're in the project directory
import os
if os.path.exists('/content/comic-gen'):
    os.chdir('/content/comic-gen')
    
print(f"📂 Working directory: {os.getcwd()}")
print("🤖 Using LOCAL Ollama (Llama 3.2) - No API calls!\n")

# Build command
storage_flag = f"--storage hf --hf_repo '{HF_REPO_ID}'" if USE_HF_STORAGE and os.environ.get('HF_TOKEN') else "--storage local"

# Run the main script
!python src/main.py --input "$INPUT_FILE" --output "$OUTPUT_DIR" --colab {storage_flag}

In [ ]:
# @title 4. Download Output
from google.colab import files
import os

if os.path.exists('output'):
    !zip -r comic_output.zip output
    files.download('comic_output.zip')
    print("✅ Download started!")
else:
    print("⚠️ No output folder found. Did the pipeline run successfully?")